# Design of an Object Oriented Data Model

## 

1. **Field Type Classification** – Identify each field as primitive, optional, collection, or nested.
2. **Per-Field Effort Weights** – Assign LOC and complexity contributions for each category.
3. **Estimator Refactor** – Generalize estimators to accept a list of field types.
4. **Effort Aggregation** – Accumulate weighted LOC and complexity.

### Field Type Classification


In [9]:
def classify_field_type(type_str: str) -> str:
    """
    Classify a type string into one of: primitive, optional, collection, nested.
    """
    primitive_types = {"int", "double", "bool", "String", "number"}
    
    if type_str.endswith("?"):
        return "optional"
    elif type_str in primitive_types:
        return "primitive"
    elif type_str.startswith("List") or type_str.startswith("Map"):
        return "collection"
    else:
        return "nested"


### Complexity Weights

In [10]:
FIELD_COMPLEXITY_WEIGHTS = {
    "primitive": {"loc": 1, "complexity": 1},
    "optional": {"loc": 1, "complexity": 2},
    "collection": {"loc": 2, "complexity": 3},
    "nested": {"loc": 2, "complexity": 4},
}


### Generalized Estimator Utility

We define a helper to compute weighted totals across field types:


In [11]:
from typing import List

def compute_weighted_effort(field_types: List[str], base_loc=0, base_complexity=0, scaling=1.0) -> dict:
    """
    Aggregate LOC and complexity for a list of field types, with optional base values.
    """
    total_loc = base_loc
    total_complexity = base_complexity

    for t in field_types:
        category = classify_field_type(t)
        weights = FIELD_COMPLEXITY_WEIGHTS[category]
        total_loc += scaling * weights["loc"]
        total_complexity += scaling * weights["complexity"]

    return {
        "loc": int(round(total_loc)),
        "complexity": int(round(total_complexity)),
    }


### Estimators



In [12]:

def estimate_to_string_effort_typed(field_types: List[str]) -> dict:
    return compute_weighted_effort(field_types, base_loc=0, base_complexity=1)

def estimate_equals_effort_typed(field_types: List[str]) -> dict:
    return compute_weighted_effort(field_types, base_loc=1, base_complexity=1)

def estimate_copy_with_effort_typed(field_types: List[str]) -> dict:
    return compute_weighted_effort(field_types, base_loc=2, base_complexity=2)

def estimate_validation_class_effort_typed(field_types: List[str]) -> dict:
    return compute_weighted_effort(field_types, base_loc=3, base_complexity=2, scaling=2)

def estimate_transformer_class_effort_typed(field_types: List[str]) -> dict:
    return compute_weighted_effort(field_types, scaling=2)



In [13]:
### Registry

In [14]:
typed_effort_estimators = {
    "toString": estimate_to_string_effort_typed,
    "equals": estimate_equals_effort_typed,
    "copyWith": estimate_copy_with_effort_typed,
    "validationClass": estimate_validation_class_effort_typed,
    "transformerClass": estimate_transformer_class_effort_typed,
    # more as needed...
}


### Effort Summary Function

In [15]:
def estimate_total_effort_typed(field_types: List[str], selected_functions=None) -> dict:
    if selected_functions is None:
        selected_functions = list(typed_effort_estimators.keys())

    breakdown = {}
    total_loc = 0
    total_complexity = 0

    for name in selected_functions:
        estimator = typed_effort_estimators.get(name)
        if estimator:
            result = estimator(field_types)
            breakdown[name] = result
            total_loc += result["loc"]
            total_complexity += result["complexity"]

    return {
        "total_loc": total_loc,
        "total_complexity": total_complexity,
        "breakdown": breakdown
    }



### Example Usage

In [16]:

field_types = ["int", "String", "List<int>", "Map<String, dynamic>", "User", "bool?"]
report = estimate_total_effort_typed(field_types)
report


{'total_loc': 69,
 'total_complexity': 104,
 'breakdown': {'toString': {'loc': 9, 'complexity': 15},
  'equals': {'loc': 10, 'complexity': 15},
  'copyWith': {'loc': 11, 'complexity': 16},
  'validationClass': {'loc': 21, 'complexity': 30},
  'transformerClass': {'loc': 18, 'complexity': 28}}}